In [ ]:
%run './model/multi_corpus.py'
# %run './constants.py'

import itertools

from mpl_chord_diagram import chord_diagram

In [ ]:
corpora = co_occurence_graphs(n_edges=5000)
Gs = {field_name: corpus['G'] for (field_name, corpus) in corpora.items()}
Dfs = {field_name: corpus['Df'] for (field_name, corpus) in corpora.items()}

In [ ]:
top_terms = 5

for field_name, G in Gs.items():

    print(field_name)

    # if field_name not in ['Geometry']:
    #     continue

    # communities = [frozenset(c) for c in nx.community.louvain_communities(G)]
    communities = nx.community.louvain_communities(G)

    core_threshold = 3

    cores = nx.core_number(G)
    centrality = nx.degree_centrality(G)

    G_communities = nx.Graph()
    for community_i, community_j in itertools.combinations(communities, 2):
        
        edge_boundary = nx.edge_boundary(G, community_i, community_j)
        edge_weight = sum([G[u][v]['weight'] for (u, v) in edge_boundary])

        if edge_weight:

            community_core_i = dict(filter(lambda tup: tup[0] in community_i, cores.items()))
            community_core_i = dict(filter(lambda tup: tup[1] >= max(community_core_i.values())-core_threshold, community_core_i.items()))
            community_core_i = dict(sorted(community_core_i.items(), key=lambda tup: tup[1]))

            centrality_core_i = dict(filter(lambda t: t[0] in community_core_i, centrality.items()))
            centrality_core_i = sorted(centrality_core_i.items(), key=lambda t: t[1], reverse=True)
            centrality_core_i = [term for term, _ in centrality_core_i[:top_terms]]

            community_i_name = '\n'.join(centrality_core_i)

            community_core_j = dict(filter(lambda tup: tup[0] in community_j, cores.items()))
            community_core_j = dict(filter(lambda tup: tup[1] >= max(community_core_j.values())-core_threshold, community_core_j.items()))
            community_core_j = dict(sorted(community_core_j.items(), key=lambda tup: tup[1]))

            centrality_core_j = dict(filter(lambda t: t[0] in community_core_j, centrality.items()))
            centrality_core_j = sorted(centrality_core_j.items(), key=lambda t: t[1], reverse=True)
            centrality_core_j = [term for term, _ in centrality_core_j[:top_terms]]

            community_j_name = '\n'.join(centrality_core_j)

            G_communities.add_edge(community_i_name, community_j_name, weight=edge_weight)

    nodes = G_communities.nodes
    M = nx.adjacency_matrix(G_communities).toarray()

    # np.fill_diagonal(M, 0)
    # # print(M)
    # upper_triangle = np.triu(M)
    degree_centrality = np.sum(M, axis=0) / 2
    # degree_centrality_list = degree_centrality.tolist()
    
    d = dict(sorted(zip(nodes, degree_centrality), key=lambda t: t[1], reverse=True))

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_axes([0, 0, 1, 1])

    plt.bar(d.keys(), d.values(), edgecolor='k')

    ax.set_xlabel('Community')
    ax.set_ylabel('Edges')

    plt.setp(ax.spines.values(), lw=1, color='k', alpha=1)
    fig.tight_layout()


    # fig, ax = plt.subplots(figsize=(10, 10))
    # chord_diagram(M, names=nodes, directed=True, rotate_names=True, ax=ax, fontsize=16)
    # ax.set_title(field_name, y=1.15, fontsize=20)

    # fig.savefig(f'./chord/{field_name}.png', dpi=150, transparent=True, bbox_inches='tight')

    plt.tight_layout()

    fig.savefig(f'./community_edge_weights/{field_name}.png', dpi=150, transparent=True, bbox_inches='tight')
    plt.show()